In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [8]:
profile_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 0 to 658
Data columns (total 92 columns):
Network                                    659 non-null object
School_ID                                  659 non-null int64
Legacy_Unit_ID                             659 non-null int64
Finance_ID                                 659 non-null int64
Short_Name                                 659 non-null object
Long_Name                                  659 non-null object
School_Type                                659 non-null object
Primary_Category                           659 non-null object
Is_High_School                             659 non-null object
Is_Middle_School                           659 non-null object
Is_Elementary_School                       659 non-null object
Is_Pre_School                              659 non-null object
Summary                                    656 non-null object
Administrator_Title                        659 non-null object
Administrator   

In [2]:
profile_2017 = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1617.csv')
header2 = ['Network', 'School_ID', 'School_Name', 'Total', 'White_Student_Count', 
          'White_Student_Pct', 'African_American_Count', 'African_America_Pct', 
         'AAPI_Count', 'AAPI_Pct', 'Native_American_Count', 'Native_American_Pct',
          'Hispanic_Student_Count', 'Hispanic_Student_Pct', 
          'Multi-Racial_Student_Count', 'Multi-Racial_Student_Pct', 
          'Asian_Student_Count', 'Asian_Student_Pct', 'Hawaiian_PI_Count', 
          'Hawaiian_PI_Pct', 'Not_Available_Count', 'Not_Available_Pct']
demo_data_2017 = pd.read_excel('Demographics_RacialEthnic_2017.xls', 'Schools', skiprows=2, names = header2)
demo_data_2017['School_ID'].astype(int, inplace=True);

In [3]:
profile_merge = pd.merge(demo_data_2017[['Network', 'School_ID']], profile_2017, how='inner', on='School_ID')

In [4]:
X_train, X_test = train_test_split(profile_merge, test_size=0.20)

In [5]:
grouped_1 = X_train.groupby('Network')
network_totals = grouped_1['Student_Count_Total'].sum()
black_student_totals = grouped_1['Student_Count_Black'].sum()
hispanic_student_totals = grouped_1['Student_Count_Hispanic'].sum()
white_student_totals = grouped_1['Student_Count_White'].sum()
asian_student_totals = grouped_1['Student_Count_Asian'].sum()
native_american_totals = grouped_1['Student_Count_Native_American'].sum()
hawaiian_pi_totals = grouped_1['Student_Count_Hawaiian_Pacific_Islander'].sum()
asian_pi_totals = grouped_1['Student_Count_Asian_Pacific_Islander'].sum()
other_ethnicity_totals = grouped_1['Student_Count_Other_Ethnicity'].sum()
multi_totals = grouped_1['Student_Count_Multi'].sum()
na_totals = grouped_1['Student_Count_Ethnicity_Not_Available'].sum()

In [6]:
df = {'Network_Totals': network_totals, 'Black_Student_Totals': black_student_totals,
     'Hispanic_Student_Totals': hispanic_student_totals, 'White_Student_Totals': 
      white_student_totals, 'Asian_Student_Totals': asian_student_totals, 
     'Native_American_Student_Totals': native_american_totals, 'Hawaiian_Pacific_Islander_Totals':
     hawaiian_pi_totals, 'Asian_Pacific_Islander_Totals': asian_pi_totals, 'Other_Ethnicity_Totals':
     other_ethnicity_totals, 'Multi_Ethnicity_Totals': multi_totals, 
      'Ethnicity_NA_Totals': na_totals}
df = pd.DataFrame(df)

In [7]:
df.drop('Other_Ethnicity_Totals', axis=1, inplace=True)
df['Totals-Ethnicity_NA'] = df['Network_Totals']-df['Ethnicity_NA_Totals']
df.drop('Ethnicity_NA_Totals', axis=1, inplace=True)

In [8]:
row_total = []
test = df['Totals-Ethnicity_NA']
for n in range(len(df.index)):
    network_total = df.iloc[n, 9]
    x = df.iloc[n, 1:9]
    row_total.append(sum([(j/network_total)*np.log(1/(j/network_total)) for j in x if j!=0]))
df['Network_Entropy'] = np.array(row_total)

In [9]:
X_train['Network_Entropy'] = X_train['Network']
network_1 = df.loc['Network 1', 'Network_Entropy']
network_2 = df.loc['Network 2', 'Network_Entropy']
network_3 = df.loc['Network 3', 'Network_Entropy']
network_4 = df.loc['Network 4', 'Network_Entropy']
network_5 = df.loc['Network 5', 'Network_Entropy']
network_6 = df.loc['Network 6', 'Network_Entropy']
network_7 = df.loc['Network 7', 'Network_Entropy']
network_8 = df.loc['Network 8', 'Network_Entropy']
network_9 = df.loc['Network 9', 'Network_Entropy']
network_10 = df.loc['Network 10', 'Network_Entropy']
network_11 = df.loc['Network 11', 'Network_Entropy']
network_12 = df.loc['Network 12', 'Network_Entropy']
network_13 = df.loc['Network 13', 'Network_Entropy']
ausl = df.loc['AUSL', 'Network_Entropy']
charter = df.loc['Charter', 'Network_Entropy']
contract = df.loc['Contract', 'Network_Entropy']
isp = df.loc['ISP', 'Network_Entropy']
options = df.loc['Options', 'Network_Entropy']
service = df.loc['Service Leadership Academies', 'Network_Entropy']

mapdict = {'Network 1': network_1, 'Network 2': network_2, 'Network 3': network_3,
          'Network 4': network_4, 'Network 5': network_5, 'Network 6': network_6,
          'Network 7': network_7, 'Network 8': network_8, 'Network 9': network_9, 
          'Network 10': network_10, 'Network 11': network_11, 'Network 12': network_12,
          'Network 13': network_13, 'AUSL': ausl, 'Charter': charter, 'Contract': contract, 
          'ISP': isp, 'Options':options, 'Service Leadership Academies': service}
X_train['Network_Entropy'] = X_train['Network_Entropy'].map(mapdict)

/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
X_train.drop('Student_Count_Other_Ethnicity', axis=1, inplace=True)
X_train['Adjusted_Counts'] = X_train['Student_Count_Total'] - X_train['Student_Count_Ethnicity_Not_Available']
X_train.drop('Student_Count_Ethnicity_Not_Available', axis=1, inplace=True)

/home/neha/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
row_total = []
for i in range(len(X_train.index)):
    school_total = X_train.iloc[i, 91]
    x = X_train.iloc[i, 36:44]
    row_total.append(sum([(i/school_total)*np.log(1/(i/school_total)) for i in x if i!=0]))

In [12]:
X_train['School_Entropy'] = np.array(row_total)

/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
X_train['District_Student_Count'] = X_train['Network']
count_1 = df.loc['Network 1', 'Totals-Ethnicity_NA']
count_2 = df.loc['Network 2', 'Totals-Ethnicity_NA']
count_3 = df.loc['Network 3', 'Totals-Ethnicity_NA']
count_4 = df.loc['Network 4', 'Totals-Ethnicity_NA']
count_5 = df.loc['Network 5', 'Totals-Ethnicity_NA']
count_6 = df.loc['Network 6', 'Totals-Ethnicity_NA']
count_7 = df.loc['Network 7', 'Totals-Ethnicity_NA']
count_8 = df.loc['Network 8', 'Totals-Ethnicity_NA']
count_9 = df.loc['Network 9', 'Totals-Ethnicity_NA']
count_10 = df.loc['Network 10', 'Totals-Ethnicity_NA']
count_11 = df.loc['Network 11', 'Totals-Ethnicity_NA']
count_12 = df.loc['Network 12', 'Totals-Ethnicity_NA']
count_13 = df.loc['Network 13', 'Totals-Ethnicity_NA']
count_ausl = df.loc['AUSL', 'Totals-Ethnicity_NA']
count_charter = df.loc['Charter', 'Totals-Ethnicity_NA']
count_contract = df.loc['Contract', 'Totals-Ethnicity_NA']
count_isp = df.loc['ISP', 'Totals-Ethnicity_NA']
count_options = df.loc['Options', 'Totals-Ethnicity_NA']
service = df.loc['Service Leadership Academies', 'Totals-Ethnicity_NA']

mapdict_count = {'Network 1': count_1, 'Network 2': count_2, 'Network 3': count_3,
          'Network 4': count_4, 'Network 5': count_5, 'Network 6': count_6,
          'Network 7': count_7, 'Network 8': count_8, 'Network 9': count_9, 
          'Network 10': count_10, 'Network 11': count_11, 'Network 12': count_12,
          'Network 13': count_13, 'AUSL': count_ausl, 'Charter': count_charter, 'Contract': count_contract, 
          'ISP': count_isp, 'Options': count_options, 'Service Leadership Academies':service}
X_train['District_Student_Count'] = X_train['District_Student_Count'].map(mapdict_count)

/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
X_train['School_Theil'] = (X_train['Adjusted_Counts']*(X_train['Network_Entropy']-X_train['School_Entropy'])
                    /(X_train['Network_Entropy']*X_train['District_Student_Count']))

/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
X_train

,Network,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,...,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location,Network_Entropy,Adjusted_Counts,School_Entropy,District_Student_Count,School_Theil
244,Network 8,609829,2540,22481,BURROUGHS,John C Burroughs Elementary School,Neighborhood,ES,N,Y,...,NaN,NaN,NaN,NaN,"3542 S WASHTENAW AVE\nChicago, Illinois 60632\...",0.456511,476,0.258215,15336,0.013482
207,Network 7,610383,7600,55171,SOCIAL JUSTICE HS,Greater Lawndale High School For Social Justice,Small,HS,Y,N,...,NaN,NaN,NaN,NaN,"3120 S KOSTNER AVE\nChicago, Illinois 60623\n(...",0.226495,283,0.402382,11788,-0.018643
6,Network 1,610515,8045,26921,DISNEY II ES,Disney II Magnet School,Magnet,ES,N,N,...,NaN,NaN,NaN,NaN,"3815 N KEDVALE AVE\nChicago, Illinois 60641\n(...",1.190994,376,1.285450,31130,-0.000958
449,AUSL,610053,4620,24341,MARQUETTE,Marquette Elementary School,Neighborhood,ES,N,Y,...,NaN,NaN,NaN,NaN,"6550 S RICHMOND ST\nChicago, Illinois 60629\n(...",0.837081,1169,0.737100,12912,0.010814
232,Network 8,610317,7470,26451,BRIGHTON PARK,Brighton Park Elementary School,Neighborhood,ES,N,Y,...,NaN,NaN,NaN,NaN,"3825 S WASHTENAW AVE\nChicago, Illinois 60632\...",0.456511,453,0.313614,15336,0.009246
536,Charter,400033,2420,66061,CICS - LONGWOOD,CICS - Longwood,Charter,HS,Y,Y,...,NaN,NaN,NaN,NaN,"1309 W 95TH ST\nChicago, Illinois 60643\n(41.7...",0.870355,1321,0.068860,41053,0.029632
461,AUSL,609727,1510,46261,PHILLIPS HS,Wendell Phillips Academy High School,Neighborhood,HS,Y,N,...,NaN,NaN,NaN,NaN,"244 E PERSHING RD\nChicago, Illinois 60653\n(4...",0.837081,637,0.079139,12912,0.044670
268,Network 9,609713,1390,46171,HYDE PARK HS,Hyde Park Academy High School,Neighborhood,HS,Y,N,...,NaN,NaN,NaN,NaN,"6220 S STONY ISLAND AVE\nChicago, Illinois 606...",0.300278,852,0.089165,10415,0.057514
596,Charter,400075,2600,66321,U OF C - DONOGHUE,University of Chicago - Donoghue,Charter,ES,N,N,...,NaN,NaN,NaN,NaN,"707 E 37TH ST\nChicago, Illinois 60653\n(41.82...",0.870355,525,0.092595,41053,0.011428
360,Network 11,610153,5670,25191,RYDER,William H Ryder Math & Science Specialty ES,Neighborhood,ES,N,Y,...,NaN,NaN,NaN,NaN,"8716 S WALLACE ST\nChicago, Illinois 60620\n(4...",0.309821,397,0.000000,13168,0.030149


In [16]:
y_train = (X_train['Adjusted_Counts']*(X_train['Network_Entropy']-X_train['School_Entropy'])
                    /(X_train['Network_Entropy']*X_train['District_Student_Count']))

In [17]:
dropped = X_train.copy()
to_drop = ['Long_Name', 'Grades_Offered_All', 'Statistics_Description', 'Demographic_Description', 'School_Hours', 'Freshman_Start_End_Time', 'After_School_Hours', 'Earliest_Drop_Off_Time', 'Classification_Description', 'Third_Contact_Title', 'Third_Contact_Name', 'Fourth_Contact_Title', 'Fourth_Contact_Name', 'Fifth_Contact_Title', 'Fifth_Contact_Name', 'Sixth_Contact_Title', 'Sixth_Contact_Name', 'Seventh_Contact_Title', 'Seventh_Contact_Name', 'Rating_Statement', 'School_Year', 'Significantly_Modified', 'Adjusted_Counts', 'Transportation_Bus', 'Transportation_El', 'Transportation_Metra', 'School_ID', 'Legacy_Unit_ID', 'Finance_ID', 'Short_Name', 'Primary_Category', 'Summary', 'Administrator_Title', 'Administrator', 'Secondary_Contact_Title', 'Secondary_Contact', 'Address', 'City', 'State', 'Zip', 'Phone', 'Fax', 'CPS_School_Profile', 'Website', 'Facebook', 'Twitter', 'Youtube', 'Pinterest', 'Attendance_Boundaries', 'Grades_Offered']
dropped.drop(to_drop, axis=1, inplace=True)

In [18]:
dropped['PreK_School_Day'] = dropped['PreK_School_Day'].fillna(0)
dropped['Kindergarten_School_Day'] = dropped['Kindergarten_School_Day'].fillna(0)
dropped['Classroom_Languages'] = dropped['Classroom_Languages'].fillna(0)
dropped['Classroom_Languages'] = np.where(dropped['Classroom_Languages'] != 0, 1, 0)
idx = dropped.index[~dropped['Classroom_Languages'].isnull()]
dropped['Classroom_Languages'].loc[idx]  = 1
dropped['Classroom_Languages'].fillna(0, inplace=True)
dropped['Bilingual_Services'].fillna(0, inplace=True)
dropped['Refugee_Services'].fillna(0, inplace=True)
dropped['Title_1_Eligible'].fillna(0, inplace=True)
# dropped['PreSchool_Inclusive'] = np.where(dropped['PreSchool_Inclusive'] == 'Y', 1, 0)
# dropped['Preschool_Instructional'] = np.where(dropped['Preschool_Instructional'] == 'Y', 1, 0)
dropped['Hard_Of_Hearing'].fillna('False', inplace=True)
dropped['Visual_Impairments'].fillna('False', inplace=True)
dropped['PreSchool_Inclusive'] = np.where(dropped['PreSchool_Inclusive'] == 'Y', 1, 0)
dropped['Preschool_Instructional'] = np.where(dropped['Preschool_Instructional'] == 'Y', 1, 0)
dropped['College_Enrollment_Rate_School'].unique()

/home/neha/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


array([ nan, 56.6, 64.9, 55.4, 47.7, 88.4, 63. ,  5.1, 93.4, 77.2, 44.8,
       34.4, 14.9, 61.4, 49.4, 26.4, 35.9, 43.5, 54.4, 65.8, 12.5, 58.3,
       19.4, 36.2, 20.8,  7.4, 41.2, 27. , 80. , 78.3, 45.7, 58.5, 62.2,
       22.4, 50.1,  8.3, 47.2, 45.3, 68. ,  0. , 18.2, 43.3,  3.7, 46.2,
       41. , 54.9, 84.7, 47.4, 40. ,  6.3, 37.8, 76.9, 31.7, 39.3, 45.2,
       59.4, 51.1, 81.6, 72.7, 56.2,  8.9,  7.8, 54.6, 18.8,  4.2, 38. ,
       44.6, 81.3, 80.4, 31.9, 40.5, 50.7, 16.7, 12.8, 14.6, 84.2, 85.1,
       40.4, 54.1, 64.3, 87.7, 76.6, 20.7, 51.5, 66.4, 62.3, 65.4, 65.7,
        6.8,  2.1, 57.8, 76.7, 19.1, 67.1, 84.4, 56.3, 87.3, 85.2, 74.6,
       63.3, 61.5, 65.5, 72.6, 79.6,  6.9, 63.2, 88. , 83.6, 59.6, 79.8,
       59.5, 22.2, 24.4, 75. , 64.2, 50.6, 21.8, 51. , 73.4, 60. , 74.7,
       43.8, 82.1, 53.2, 66.8])

In [19]:
n_1 = dropped.loc[dropped.index[dropped['Network']=='Network 1']]['College_Enrollment_Rate_School'].notnull().sum()
mean_1 = dropped[dropped['Network'] == 'Network 1']['College_Enrollment_Rate_School'].sum()/n_1

n_2 = dropped.loc[dropped.index[dropped['Network']=='Network 2']]['College_Enrollment_Rate_School'].notnull().sum()
mean_2 = dropped[dropped['Network'] == 'Network 2']['College_Enrollment_Rate_School'].sum()/n_2

n_3 = dropped.loc[dropped.index[dropped['Network']=='Network 3']]['College_Enrollment_Rate_School'].notnull().sum()
mean_3 = dropped[dropped['Network'] == 'Network 3']['College_Enrollment_Rate_School'].sum()/n_3

n_4 = dropped.loc[dropped.index[dropped['Network']=='Network 4']]['College_Enrollment_Rate_School'].notnull().sum()
mean_4 = dropped[dropped['Network'] == 'Network 4']['College_Enrollment_Rate_School'].sum()/n_4

n_5 = dropped.loc[dropped.index[dropped['Network']=='Network 5']]['College_Enrollment_Rate_School'].notnull().sum()
mean_5 = dropped[dropped['Network'] == 'Network 5']['College_Enrollment_Rate_School'].sum()/n_5

n_6 = dropped.loc[dropped.index[dropped['Network']=='Network 6']]['College_Enrollment_Rate_School'].notnull().sum()
mean_6 = dropped[dropped['Network'] == 'Network 6']['College_Enrollment_Rate_School'].sum()/n_6

n_7 = dropped.loc[dropped.index[dropped['Network']=='Network 7']]['College_Enrollment_Rate_School'].notnull().sum()
mean_7 = dropped[dropped['Network'] == 'Network 7']['College_Enrollment_Rate_School'].sum()/n_7

n_8 = dropped.loc[dropped.index[dropped['Network']=='Network 8']]['College_Enrollment_Rate_School'].notnull().sum()
mean_8 = dropped[dropped['Network'] == 'Network 8']['College_Enrollment_Rate_School'].sum()/n_8

n_9 = dropped.loc[dropped.index[dropped['Network']=='Network 9']]['College_Enrollment_Rate_School'].notnull().sum()
mean_9 = dropped[dropped['Network'] == 'Network 9']['College_Enrollment_Rate_School'].sum()/n_9

n_10 = dropped.loc[dropped.index[dropped['Network']=='Network 10']]['College_Enrollment_Rate_School'].notnull().sum()
mean_10 = dropped[dropped['Network'] == 'Network 10']['College_Enrollment_Rate_School'].sum()/n_10

n_11 = dropped.loc[dropped.index[dropped['Network']=='Network 11']]['College_Enrollment_Rate_School'].notnull().sum()
mean_11 = dropped[dropped['Network'] == 'Network 11']['College_Enrollment_Rate_School'].sum()/n_11

n_12 = dropped.loc[dropped.index[dropped['Network']=='Network 12']]['College_Enrollment_Rate_School'].notnull().sum()
mean_12 = dropped[dropped['Network'] == 'Network 12']['College_Enrollment_Rate_School'].sum()/n_12

n_13 = dropped.loc[dropped.index[dropped['Network']=='Network 13']]['College_Enrollment_Rate_School'].notnull().sum()
mean_13 = dropped[dropped['Network'] == 'Network 13']['College_Enrollment_Rate_School'].sum()/n_13

n_AUSL = dropped.loc[dropped.index[dropped['Network']=='Network AUSL']]['College_Enrollment_Rate_School'].notnull().sum()
mean_AUSL = dropped[dropped['Network'] == 'Network AUSL']['College_Enrollment_Rate_School'].sum()/n_AUSL

n_service = dropped.loc[dropped.index[dropped['Network']=='Service Leadership Academies']]['College_Enrollment_Rate_School'].notnull().sum()
mean_service = dropped[dropped['Network'] == 'Service Leadership Academies']['College_Enrollment_Rate_School'].sum()/n_service

n_isp = dropped.loc[dropped.index[dropped['Network']=='ISP']]['College_Enrollment_Rate_School'].notnull().sum()
mean_isp = dropped[dropped['Network'] == 'ISP']['College_Enrollment_Rate_School'].sum()/n_isp

n_charter = dropped.loc[dropped.index[dropped['Network']=='Charter']]['College_Enrollment_Rate_School'].notnull().sum()
mean_charter = dropped[dropped['Network'] == 'Charter']['College_Enrollment_Rate_School'].sum()/n_charter

n_contract = dropped.loc[dropped.index[dropped['Network']=='Contract']]['College_Enrollment_Rate_School'].notnull().sum()
mean_contract = dropped[dropped['Network'] == 'Contract']['College_Enrollment_Rate_School'].sum()/n_contract

n_options = dropped.loc[dropped.index[dropped['Network']=='Options']]['College_Enrollment_Rate_School'].notnull().sum()
mean_options = dropped[dropped['Network'] == 'Options']['College_Enrollment_Rate_School'].sum()/n_options

/home/neha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


In [30]:
mean_10

59.275000000000006

In [28]:
dropped.loc[dropped['Network']=='Network 10'] = dropped.loc[dropped.index[dropped['Network']=='Network 10']]['College_Enrollment_Rate_School'].fillna({'Network 10': mean_10}, inplace=True)

In [33]:
dropped.loc[dropped.index[dropped['Network']=='Network 10']]

,Network,School_Type,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,...,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Location,Network_Entropy,School_Entropy,District_Student_Count,School_Theil
